[View in Colaboratory](https://colab.research.google.com/github/hanshika0292/EIP-Assignments/blob/master/HANSHIKA_BATCH_4_ASSIGNMENT4B.ipynb)

In [2]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 50
l = 250
num_filter = 24
compression = 0.5
dropout_rate = 0.2

In [6]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 30s 0us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 24
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [11]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 24)   648         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 24)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 24)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (


Trainable params: 441,610
Non-trainable params: 9,048
__________________________________________________________________________________________________


In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [13]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
43776/50000 [=========================>....] - ETA: 41s - loss: 1.5379 - acc: 0.4333

50000/50000 [==============================] - 353s 7ms/step - loss: 1.5036 - acc: 0.4462 - val_loss: 1.3508 - val_acc: 0.5376
Epoch 2/50
26240/50000 [==============>...............] - ETA: 2:34 - loss: 1.1519 - acc: 0.5849

50000/50000 [==============================] - 342s 7ms/step - loss: 1.0975 - acc: 0.6068 - val_loss: 1.2752 - val_acc: 0.5834
Epoch 3/50
19584/50000 [==========>...................] - ETA: 3:18 - loss: 0.9459 - acc: 0.6598

50000/50000 [==============================] - 344s 7ms/step - loss: 0.9201 - acc: 0.6724 - val_loss: 1.0891 - val_acc: 0.6446
Epoch 4/50
17024/50000 [=========>....................] - ETA: 3:35 - loss: 0.8444 - acc: 0.7012

50000/50000 [==============================] - 344s 7ms/step - loss: 0.8198 - acc: 0.7105 - val_loss: 1.1351 - val_acc: 0.6629
Epoch 5/50
16064/50000 [========>.....................] - ETA: 3:44 - loss: 0.7605 - acc: 0.7284

...

50000/50000 [==============================] - 344s 7ms/step - loss: 0.2188 - acc: 0.9221 - val_loss: 0.6168 - val_acc: 0.8450
Epoch 47/50
15360/50000 [========>.....................] - ETA: 3:46 - loss: 0.2149 - acc: 0.9226

50000/50000 [==============================] - 344s 7ms/step - loss: 0.2214 - acc: 0.9208 - val_loss: 0.5257 - val_acc: 0.8592
Epoch 48/50
15360/50000 [========>.....................] - ETA: 3:46 - loss: 0.2133 - acc: 0.9249

50000/50000 [==============================] - 344s 7ms/step - loss: 0.2137 - acc: 0.9232 - val_loss: 0.7084 - val_acc: 0.8233
Epoch 49/50
15360/50000 [========>.....................] - ETA: 3:46 - loss: 0.1987 - acc: 0.9304

50000/50000 [==============================] - 343s 7ms/step - loss: 0.2152 - acc: 0.9224 - val_loss: 0.6219 - val_acc: 0.8441
Epoch 50/50
15360/50000 [========>.....................] - ETA: 3:46 - loss: 0.1946 - acc: 0.9327

50000/50000 [==============================] - 343s 7ms/step - loss: 0.2044 - acc: 0.9277 - val_loss: 0.5473 - val_acc: 0.8541


In [17]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 21s 2ms/step
Test loss: 0.5472558197021484
Test accuracy: 0.8541


In [18]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')